In [ ]:
EXPERIMENT_ID = "adhoc-now_2019"

In [ ]:
%load_ext autoreload
%autoreload 2

import pathlib

import pandas as pd
pd.set_option('display.float_format', lambda x: '%.1f' % x)

import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams["font.size"] = "20"

from data_handlers.network import parse_bwms
from data_handlers.procmem import parse_pidstat

In [ ]:
experiment_path = pathlib.Path(f"/maci_data/binary_files/{EXPERIMENT_ID}/")

In [ ]:
network_df = parse_bwms(experiment_path)

In [ ]:
procmem_df = parse_pidstat(experiment_path)

In [ ]:
df_groups = ['id', 'software', 'size', 'node', 'node_num', 'dt']

In [ ]:
clean_network = network_df.groupby(['id', 'software', 'size', 'node_num', 'dt']).sum()
clean_network['Mbit/s'] = clean_network['bytes_out/s'] / 1024 / 1024 * 8

In [ ]:
clean_procmem = procmem_df.groupby(['id', 'software', 'size', 'node_num', 'dt']).sum()
clean_procmem = clean_procmem[['%CPU', 'RSS']]

In [ ]:
merged = clean_network.merge(clean_procmem, left_index=True, right_index=True)

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=2)
plt.subplots_adjust(wspace=.1, hspace=.3)

fig.set_size_inches(15,10)

color_mbit = 'C0'
color_cpu = 'C1'

for ax, (name, software_df) in zip(list(axs[0]) + list(axs[1]), merged.groupby(['software'])):
    ax.tick_params(axis='y', labelcolor=color_mbit)
    ax.set_xlim(-10, 210)
    ax.set_ylim(0.01, 101)
    ax.set_title(name)
    
    ax2 = ax.twinx()
    ax2.tick_params(axis='y', labelcolor=color_cpu)
    ax2.set_ylim(0.01, 200)
    
    for size, size_df in software_df.groupby(['size']):
        if size == 26214400:
            for nodes, nodes_df in size_df.groupby(['node_num']):
                if nodes == 32:
                    
                    nodes_df = nodes_df.reset_index()
                    
                    sns.lineplot(
                        x=nodes_df['dt'],
                        y=nodes_df['Mbit/s'].rolling(2).mean(),
                        ax=ax,
                        color=color_mbit
                    )
                    
                    sns.lineplot(
                        x=nodes_df['dt'],
                        y=nodes_df['%CPU'].rolling(2).mean(),
                        ax=ax2,
                        color=color_cpu
                    )
        
                    ax.set_ylabel('Mbit/s', color=color_mbit)
                    if name in ['Forban', 'Serval']:
                        ax.set_ylabel("")
                        ax.get_yaxis().set_ticks([])

                    if name in ['DTN7', 'Forban']:
                        ax.set_xlabel("")


                    ax2.set_ylabel('%CPU', color=color_cpu)
                    if name in ['DTN7', 'IBRDTN']:
                        ax2.set_ylabel("")
                        ax2.get_yaxis().set_ticks([])

                    if name in ['IBRDTN', 'Serval']:
                        ax.set_xlabel("Time (s)")
                        
                    if name == 'IBRDTN':
                        ax.set_title('IBR-DTN')
                        
plt.savefig('plots/cpu_network.pdf')